In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image
import os
import numpy as np
import tensorflow as tf
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt 
from keras.preprocessing import image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
import os
import soundfile as sf
import numpy as np
import librosa
from skimage.transform import resize
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold






mel_train = []
cqt_train = []
mel_test = []
cqt_test = []
label_train = []
label_test = []

In [ ]:
def load_images_from_path(path, label):
    images = []
    labels = []

    for file in os.listdir(path):
        images.append(tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(os.path.join(path, file), target_size=(224, 224, 3))))
        labels.append((label))
        
    return images, labels

In [ ]:
images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/train/pain', 0)
    
mel_train += images
label_train += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/train/burping', 1)
    
mel_train += images
label_train += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/train/discomfort', 2)
    
mel_train += images
label_train += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/train/hungry', 3)
    
mel_train += images
label_train += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/train/tired', 4)
    
mel_train += images
label_train += labels

In [ ]:
images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/test/pain', 0)
    
mel_test += images
label_test += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/test/burping', 1)
    
mel_test += images
label_test += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/test/discomfort', 2)
    
mel_test += images
label_test += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/test/hungry', 3)
    
mel_test += images
label_test += labels

images, labels = load_images_from_path('/content/drive/MyDrive/spectro_aug_tt/test/tired', 4)
    
mel_test += images
label_test += labels

In [ ]:

def load_images_from_path_1(path):
    images = []
    
    for root, _, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            images.append(tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(file_path, target_size=(224, 224, 3))))
    
    return images

In [ ]:
images  = load_images_from_path_1('/content/drive/MyDrive/spectro_aug_cqt_tt/train')
    
cqt_train += images


In [ ]:
images  = load_images_from_path_1('/content/drive/MyDrive/spectro_aug_cqt_tt/test')

cqt_test += images

In [ ]:
from tensorflow.keras.utils import to_categorical

mel_train_norm = np.array(mel_train) / 255
mel_test_norm = np.array(mel_test) / 255
cqt_train_norm = np.array(cqt_train) / 255
cqt_test_norm = np.array(cqt_test) / 255

label_train_encoded = to_categorical(label_train)
label_test_encoded = to_categorical(label_test)

In [ ]:
mel_test_norm.shape

In [ ]:
cqt_test_norm.shape

In [ ]:
mel_train_norm.shape

In [ ]:
cqt_train_norm.shape

In [ ]:
label_train_encoded.shape

In [ ]:
label_test_encoded.shape

In [ ]:
# Define input layers
mel_input = Input(shape=(224, 224, 3))  # Modify the shape according to your mel spectrogram dimensions
cqt_input = Input(shape=(224, 224, 3))

# Convolutional layers for Mel spectrograms
mel_conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(mel_input)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Conv2D(64, kernel_size=(3, 3), activation='relu')(mel_conv)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Conv2D(128, kernel_size=(3, 3), activation='relu')(mel_conv)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Flatten()(mel_conv)

# Convolutional layers for CQT spectrograms
cqt_conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(cqt_input)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Conv2D(64, kernel_size=(3, 3), activation='relu')(cqt_conv)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Conv2D(128, kernel_size=(3, 3), activation='relu')(cqt_conv)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Flatten()(cqt_conv)

# Merge the branches
merged = concatenate([mel_conv, cqt_conv])

# Dense layers for classification
dense = Dense(256, activation='relu')(merged)
dense = Dropout(0.5)(dense)  # Add dropout with a rate of 0.5
dense = Dense(128, activation='relu')(dense)
dense = Dropout(0.5)(dense)  # Add dropout with a rate of 0.5
output = Dense(5, activation='softmax')(dense)

# Define the model
model = Model(inputs=[mel_input, cqt_input], outputs=output)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
earlystop_callback = EarlyStopping(
    monitor='val_accuracy', patience=5, mode='max', verbose=1, restore_best_weights=True
)



In [ ]:
from skimage.transform import resize

def spec_augment(spec, num_time_drop=2, num_freq_drop=2, max_time_stripes=2, max_freq_stripes=2):
    augmented_spec = spec.copy()

    # Apply time dropping
    for _ in range(num_time_drop):
        time_start = np.random.randint(0, spec.shape[0] - 1)
        time_end = np.random.randint(time_start + 1, spec.shape[0])
        augmented_spec[time_start:time_end, :] = 0

    # Apply frequency dropping
    for _ in range(num_freq_drop):
        freq_start = np.random.randint(0, spec.shape[1] - 1)
        freq_end = np.random.randint(freq_start + 1, spec.shape[1])
        augmented_spec[:, freq_start:freq_end] = 0

    # Apply time warping
    for _ in range(max_time_stripes):
        time_start = np.random.randint(0, spec.shape[0] - 1)
        time_end = np.random.randint(time_start + 1, spec.shape[0])
        time_mid = np.random.randint(time_start, time_end)
        time_mid_val = spec[time_mid, :]
        augmented_spec[time_start:time_mid, :] = augmented_spec[time_start:time_mid, :][::-1]
        augmented_spec[time_mid:time_end, :] = augmented_spec[time_mid:time_end, :][::-1]
        augmented_spec[time_start:time_end, :] = time_mid_val

    # Apply frequency warping
    for _ in range(max_freq_stripes):
        freq_start = np.random.randint(0, spec.shape[1] - 1)
        freq_end = np.random.randint(freq_start + 1, spec.shape[1])
        freq_mid = np.random.randint(freq_start, freq_end)
        freq_mid_val = spec[:, freq_mid]
        freq_mid_val = np.expand_dims(freq_mid_val, axis=1)
        freq_mid_val = np.repeat(freq_mid_val, freq_end - freq_start, axis=1)
        augmented_spec[:, freq_start:freq_end] = freq_mid_val

    augmented_spec = resize(augmented_spec, (224, 224, 3), preserve_range=True)

    return augmented_spec


In [ ]:
# Define the number of folds
n_folds = 5

# Create an instance of KFold
kf = KFold(n_splits=n_folds, shuffle=True)

fold = 1
for train_index, val_index in kf.split(mel_train_norm):
    print(f'Fold: {fold}')

    # Get the training and validation data for the current fold
    mel_train_fold = mel_train_norm[train_index]
    mel_val_fold = mel_train_norm[val_index]
    cqt_train_fold = cqt_train_norm[train_index]
    cqt_val_fold = cqt_train_norm[val_index]
    label_train_fold = label_train_encoded[train_index]
    label_val_fold = label_train_encoded[val_index]

    # Calculate the count of each class in the training set
    class_counts = np.sum(label_train_fold, axis=0)

    # Find the majority class count
    majority_class_count = np.max(class_counts)

    # Augment the minority classes to match the majority class count using SpecAugment
    for i, count in enumerate(class_counts):
        if count < majority_class_count:
            # Determine the number of SpecAugment augmentations required for the current class
            num_augmentations = int(majority_class_count - count)

            # Get the indices of the samples belonging to the current class
            class_indices = np.where(label_train_fold[:, i] == 1)[0]

            # Randomly select samples from the current class for SpecAugment augmentation
            selected_indices = np.random.choice(class_indices, size=num_augmentations, replace=True)

            # Apply SpecAugment augmentation to the selected samples for both Mel and CQT spectrograms
            augmented_mel_train = []
            augmented_cqt_train = []
            augmented_labels = []
            for idx in selected_indices:
                augmented_mel_spec = spec_augment(mel_train_fold[idx])
                augmented_cqt_spec = spec_augment(cqt_train_fold[idx])
                augmented_label = label_train_fold[idx]
                augmented_mel_train.append(augmented_mel_spec)
                augmented_cqt_train.append(augmented_cqt_spec)
                augmented_labels.append(augmented_label)
            augmented_mel_train = np.array(augmented_mel_train)
            augmented_cqt_train = np.array(augmented_cqt_train)
            augmented_labels = np.array(augmented_labels)

            # Concatenate the augmented samples with the original training data and labels
            mel_train_fold = np.concatenate((mel_train_fold, augmented_mel_train))
            cqt_train_fold = np.concatenate((cqt_train_fold, augmented_cqt_train))
            label_train_fold = np.concatenate((label_train_fold, augmented_labels))

    # Fit the model using the augmented data
    history = model.fit(
        [mel_train_fold, cqt_train_fold],
        label_train_fold,
        batch_size=32,
        epochs=100,
        validation_data=([mel_test_norm, cqt_test_norm], label_test_encoded),
        callbacks=[earlystop_callback]
    )
    



    fold += 1


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred = model.predict([mel_test_norm, cqt_test_norm])

# Convert the predictions from one-hot encoded format to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(label_test_encoded, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_true_labels, y_pred_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')


# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)




In [ ]:


from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
confusion_mat = confusion_matrix(y_true_labels, y_pred_labels)

# Calculate accuracy scores for each class
class_accuracy = confusion_mat.diagonal() / confusion_mat.sum(axis=1)

# Print the accuracy scores for each class
for i, acc in enumerate(class_accuracy):
    print(f"Accuracy for class {i}: {acc}")


In [ ]:
import matplotlib.pyplot as plt

# Get the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot the training curve
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(12, 4))

# Plot the loss curve
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot the accuracy curve
plt.subplot(1, 2, 2)
plt.plot(epochs, train_acc, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
